# LSTM5d - NSGA II
## (Modelo 2)

In [6]:
# ────────────────────────────────────────────────────────────────────────────────
# 0. LIBRERÍAS BÁSICAS Y RUTAS
# ────────────────────────────────────────────────────────────────────────────────
import pandas as pd, numpy as np, joblib, yfinance as yf, pathlib, sys, warnings
from scipy.stats import skew, kurtosis
warnings.simplefilter("ignore", FutureWarning)

ROOT = pathlib.Path().resolve().parent.parent        #  …/notebooks → raíz del repo
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from src import config as cfg                        # cfg.DATA, cfg.RESULT …

# ────────────────────────────────────────────────────────────────────────────────
# 1. CALENDARIO DIARIO MAESTRO  (S&P 500 como referencia de días hábiles)
# ────────────────────────────────────────────────────────────────────────────────
START = "2019-01-02"
END   = pd.Timestamp.today().normalize()

gspc     = yf.download("^GSPC", start=START, end=END,
                       progress=False, auto_adjust=False)
price_gs = gspc["Adj Close"] if "Adj Close" in gspc.columns else gspc["Close"]
calendar = price_gs.dropna().index                  # índice de fechas diario

# ────────────────────────────────────────────────────────────────────────────────
# 2. RETORNOS DEL MODELO LSTM-5d  (rellenamos huecos con 0 %)
# ────────────────────────────────────────────────────────────────────────────────
bt_path  = cfg.RESULT / "backtest_lstm5d.pkl"
bt       = joblib.load(bt_path)                     # dict con clave "retorno"
ret_sys  = bt["retorno"].reindex(calendar).fillna(0.0)   # % diario

# ────────────────────────────────────────────────────────────────────────────────
# 3. RETORNOS DE BENCHMARKS SPY / BIL  (mismo calendario)
# ────────────────────────────────────────────────────────────────────────────────
tickers   = ["SPY", "BIL"]
raw_bench = yf.download(tickers, start=START, end=END,
                        progress=False, auto_adjust=False)
prices_b  = (raw_bench["Adj Close"]
             if "Adj Close" in raw_bench.columns.get_level_values(0)
             else raw_bench["Close"]).ffill()

ret_bench = np.log(prices_b / prices_b.shift(1)).reindex(calendar).dropna()
ret_spy   = ret_bench["SPY"]
ret_cash  = ret_bench["BIL"]

# ────────────────────────────────────────────────────────────────────────────────
# 4. FUNCIÓN DE MÉTRICAS COHERENTE
# ────────────────────────────────────────────────────────────────────────────────
def stats(r):
    ann = np.sqrt(252)
    w   = (1 + r).cumprod()
    return {
        "Rentabilidad (%)" : (w.iloc[-1] - 1) * 100,
        "Volatilidad (%)"  : r.std(ddof=1) * ann * 100,
        "Sharpe"           : r.mean() / r.std(ddof=1) * ann,
        "MaxDD (%)"        : (w.cummax() - w).max() * 100,
        "Asimetría"        : skew(r),
        "Curtosis"         : kurtosis(r)
    }

# ────────────────────────────────────────────────────────────────────────────────
# 5. TABLA FINAL
# ────────────────────────────────────────────────────────────────────────────────
tabla = pd.DataFrame({
            "LSTM-5d": stats(ret_sys),
            "SPY"    : stats(ret_spy),
            "BIL"    : stats(ret_cash)
        }).T.round(2)

print("\n── Métricas 2019-hoy  (calendario diario coherente) ──\n")
display(tabla)



── Métricas 2019-hoy  (calendario diario coherente) ──



,Rentabilidad (%),Volatilidad (%),Sharpe,MaxDD (%),Asimetría,Curtosis
LSTM-5d,250.49,16.35,1.26,30.68,15.86,417.18
SPY,137.39,20.37,0.76,48.90,-0.57,13.57
BIL,16.91,0.25,9.54,0.21,0.85,1.27


In [8]:

# --- Gráfico PRINCIPAL: comparación sin DCA ---
plt.figure(figsize=(12,6))
wealth_lstm_ini.plot(label='LSTM + NSGA-II sin DCA')
wealth_spy_ini.plot(label='SPY (S&P 500) sin DCA')
wealth_cash_ini.plot(label='BIL (Cash) sin DCA')
plt.title('Evolución de la cartera SIN aportaciones mensuales (DCA)')
plt.ylabel('€')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# --- Segundo gráfico: comparación con DCA ---
plt.figure(figsize=(12,6))
wealth_lstm_dca.plot(label='LSTM + NSGA-II con DCA')
wealth_spy_dca.plot(label='SPY (S&P 500) con DCA')
wealth_cash_dca.plot(label='BIL (Cash) con DCA')
plt.title('Evolución de la cartera CON aportaciones mensuales (DCA)')
plt.ylabel('€')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [10]:
# 8. HISTOGRAMA DE RETORNOS
plt.figure(figsize=(8,4))
ret_sys.hist(bins=50, density=True, alpha=0.6, label="Sistema")
ret_sys.plot(kind="kde", label="KDE")
plt.title("Distribución de retornos diarios del sistema")
plt.xlabel("Retorno diario")
plt.grid(True)
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [12]:

# 9. DRAWDOWN PLOT
cum_returns = (1 + ret_sys).cumprod()
rolling_max = cum_returns.cummax()
drawdown = cum_returns / rolling_max - 1

plt.figure(figsize=(10,3))
drawdown.plot()
plt.title("Drawdown acumulado del sistema")
plt.ylabel("Drawdown")
plt.grid()
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [14]:
# 10. HEATMAP DE PESOS (si w_star está disponible)
df_w = pd.DataFrame(
    [r["w_star"] for r in res_df.dropna().to_dict(orient='records')],
    index=res_df.dropna().index,
    columns=tickers
)

plt.figure(figsize=(14,6))
sns.heatmap(df_w.T, cmap="viridis", cbar=True)
plt.title("Asignación de pesos a lo largo del tiempo")
plt.xlabel("Fecha")
plt.ylabel("Activo")
plt.tight_layout()
plt.show()

NameError: name 'res_df' is not defined

In [16]:
# 11. RETORNOS POR REBALANCEO
plt.figure(figsize=(12, 5))
res_df["retorno"].plot(marker='o', linestyle='-')
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.title("Retornos por rebalanceo del portafolio")
plt.ylabel("Retorno")
plt.xlabel("Fecha")
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined